## Preprocesado del conjunto de Letras - V2

En este fichero se explica el preprocesado de los datos para la creación de modelos de reconocimiento de lenguaje de signos para letras.

Esta es la segunda versión del preprocesado de los datos, en esta versión la diferencia es la procedencia del conjunto de validación que en este caso se obtendrá del conjunto de test.

En primer lugar, cargamos los ficheros con los conjuntos. Desde Kaggle ya tenemos el dataset separado en train y en test.

In [1]:
import pandas as pd
import numpy as np
import keras as keras
trainFile_letras = "../DatasetsIAO/Letras/sign_mnist_train.csv"
testFile_letras = "../DatasetsIAO/Letras/sign_mnist_test.csv"
path_npy = "../DatasetsIAO/Letras/"

letras_train = pd.read_csv(trainFile_letras)
letras_test = pd.read_csv(testFile_letras)

Using TensorFlow backend.
/home/javier/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/javier/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/javier/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/javier/.local/lib/python3.6/site-packages/tensorflow/python

Tras cargar los datos, imprimimos sus tamaños y una parte del mismo para comprobar que se han leído correctamente.

In [2]:
print(letras_train.shape)
print(letras_test.shape)

(27455, 785)
(7172, 785)


In [3]:
letras_train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


Ya que no tenemos conjunto de validación, necesitamos obtenerlo de alguno de los conjuntos que tenemos. Tras haber probado los primeros conjuntos (de la versión 1 de preprocesado) con los dos primeros modelos, sacar el conjunto de validación del conjunto de train no ha sido buena idea ya que en la página de kaggle se indica que el conjunto de train se ha aumentado haciendo cambios en las imagenes.

Es decir, según se explica en el repositorio los conjuntos se han construido de la siguiente manera:

Se obtuvieron 1704 imágenes que han sido aumentadas aplicando diferentes filtros, pixelacion aleatoria y grados de rotación, así se consiguieron obtener muchísimas más imágenes quedando en un total de 27000 imágenes de train y 7000 de test. Suponemos que antes de hacer este aumento, separaron en train y test, ya que de hacerlo al reves, se tendrían fotos en test que se encuentran de forma muy parecida en training.

Al hacer el conjunto de validación cortando el de train, se podría provocar que se evalúe la red en validación con imágenes muy parecidas a las que se tienen en train, haciendo que el porcentaje de acierto de validación sea igual queen train o incluso superior.

Dentro del conjunto de test ocurre lo mismo, solo que hacerlo así nos permite tener más rango de decisión a la hora de cambiar parámetros (ya que al menos difieren de las imágenes de train). Lo realmente correcto sería tener conjuntos disjuntos que realmente prueben la generalización de la red, pero este será un tema del que se hablará en mejoras futuras y problemas.

En esta versión, lo que hacemos es partir el conjunto de test en dos partes (aleatorizándolo antes):

In [4]:
from sklearn.utils import shuffle
# Aleatorizamos las imagenes
letras_test = shuffle(letras_test)

#Se toma la mitad de test
letras_val = letras_test[:3586]
letras_test = letras_test[3586:]

In [5]:
letras_train_Y = letras_train['label']
letras_train_X = letras_train.drop('label', axis=1)

letras_test_Y = letras_test['label']
letras_test_X = letras_test.drop('label', axis=1)

letras_val_Y = letras_val['label']
letras_val_X = letras_val.drop('label', axis=1)

In [6]:
letras_train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [7]:
letras_val

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
4143,13,195,195,196,195,195,195,196,198,197,...,104,83,123,219,212,230,216,157,106,122
2213,6,204,206,209,210,211,214,214,215,214,...,237,236,235,233,232,232,232,229,224,220
4208,23,153,156,160,163,166,169,171,173,174,...,49,46,45,39,29,32,37,38,36,22
5989,7,142,153,161,167,172,174,176,177,180,...,175,171,168,166,163,160,157,152,145,121
962,11,168,170,172,174,174,175,176,177,178,...,208,209,209,208,210,184,124,113,107,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,21,164,169,174,174,177,181,184,187,188,...,138,132,132,207,222,218,219,218,217,217
5345,14,161,164,163,164,166,166,167,167,168,...,204,203,205,203,204,204,201,202,201,201
524,18,171,172,173,173,173,174,175,175,175,...,97,75,56,144,189,190,188,187,186,185
6378,4,173,174,174,175,176,174,174,174,175,...,204,202,202,201,200,199,197,195,194,192


In [8]:
letras_test

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
1141,20,71,75,80,89,108,124,132,137,147,...,129,106,40,1,25,46,32,12,17,35
393,14,192,191,191,191,191,192,193,193,193,...,255,246,247,247,246,243,244,244,243,241
6182,8,125,130,137,141,143,146,150,154,157,...,30,51,27,9,31,48,42,34,36,43
3663,24,152,154,155,157,158,161,162,162,163,...,185,184,182,182,181,181,180,179,178,176
4503,6,195,198,200,202,204,205,206,208,207,...,220,219,217,215,212,208,208,205,202,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,15,204,178,181,161,119,203,98,48,109,...,181,172,161,165,165,166,171,138,88,69
5222,7,207,206,207,207,207,207,206,206,205,...,14,26,26,35,21,27,32,39,40,15
5762,5,178,180,182,185,187,187,187,187,188,...,216,212,210,208,209,208,207,205,205,191
4773,13,157,168,176,179,183,189,197,202,206,...,163,106,210,255,255,209,99,99,85,54


Se preparan los conjuntos para poder usarlos en la red de neuronas. En la primera posición se pone el número de imágenes, las dos siguientes dimensiones son del tamaño de la imagen y la última dimensión representan los canales de la imagen, al ser en blanco y negro, un solo canal.

Después se transforman las labels para tenerlas en one hot encoding, ya que necesitaremos tenerlas en este formato para las redes que creamos.

In [9]:
from keras.utils import np_utils

# Hacemos el reshape para que las imagenes tengan el tamaño correcto
letras_train_X = letras_train_X.values.reshape(letras_train_X.shape[0], 28, 28, 1)
letras_test_X = letras_test_X.values.reshape(letras_test_X.shape[0], 28, 28, 1)
letras_val_X = letras_val_X.values.reshape(letras_val_X.shape[0], 28, 28, 1)


# Pasamos las salidas a one hot encoding. Aunque tenemos 24 clases
# obtenemos un array de 25 posiciones. Esto se debe a que la J (numero
# 9) también se tiene en cuenta, aunque no aparezca en ningún momento.
letras_train_Y = np_utils.to_categorical(letras_train_Y)
letras_test_Y = np_utils.to_categorical(letras_test_Y)
letras_val_Y = np_utils.to_categorical(letras_val_Y)

In [10]:
print(letras_train_X.shape)
print(letras_train_Y.shape)

print(letras_test_X.shape)
print(letras_test_Y.shape)

print(letras_val_X.shape)
print(letras_val_Y.shape)

(27455, 28, 28, 1)
(27455, 25)
(3586, 28, 28, 1)
(3586, 25)
(3586, 28, 28, 1)
(3586, 25)


Y por último, guardamos los ficheros para poder usarlos.

In [11]:
np.save(path_npy + "letras_trainV2_X.npy", letras_train_X)
np.save(path_npy + "letras_trainV2_Y.npy", letras_train_Y)
np.save(path_npy + "letras_testV2_X.npy", letras_test_X)
np.save(path_npy + "letras_testV2_Y.npy", letras_test_Y)
np.save(path_npy + "letras_valV2_X.npy", letras_val_X)
np.save(path_npy + "letras_valV2_Y.npy", letras_val_Y)